# HuggingFace TimesFM test

In [1]:
# Standard librairy
import os
from pathlib import Path
from tqdm import tqdm

# Librairies
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd

# ML librairies
import torch
import torch.nn as nn

In [2]:
from transformers import (
    PatchTSTConfig,
    TimesFmModelForPrediction
)

from transformers import set_seed
set_seed(42)

In [3]:
timesfm = TimesFmModelForPrediction.from_pretrained(
    "google/timesfm-2.0-500m-pytorch",
    dtype=torch.bfloat16,
    attn_implementation="sdpa",
    device_map="auto",
)

In [6]:
print(f"\nModel: \n{timesfm}")
print("\n-----------------------\n\n")
print(f"Number of parameters: \n{timesfm.num_parameters()/1e6:.2f}M")
print("\n-----------------------\n\n")
print(f"Config: \n{timesfm.config}")


Model: 
TimesFmModelForPrediction(
  (decoder): TimesFmModel(
    (input_ff_layer): TimesFmResidualBlock(
      (input_layer): Linear(in_features=64, out_features=1280, bias=True)
      (activation): SiLU()
      (output_layer): Linear(in_features=1280, out_features=1280, bias=True)
      (residual_layer): Linear(in_features=64, out_features=1280, bias=True)
    )
    (freq_emb): Embedding(3, 1280)
    (layers): ModuleList(
      (0-49): 50 x TimesFmDecoderLayer(
        (self_attn): TimesFmAttention(
          (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (o_proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): TimesFmMLP(
          (gate_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (down_proj): Linear(in_features=1280, out_features=1280, bias=True)

In [ ]:
with torch.no_grad():
    # Passer la série through le modèle
    outputs = model.decoder(input_patches)
    
    # outputs.shape = [batch_size, num_patches, hidden_size=1280]
    
    # Option 1: Moyenne sur tous les patches
    series_embedding = outputs.mean(dim=1)  # [batch_size, 1280]